In [1]:
import data
import os, glob
import matplotlib.pyplot as plt
import cv2
import numpy as np

from networks import CNNFeatureLSTM

import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [3]:
d = data.ClogData('/ssd_icybox2_1TB/jason/clog-loss-data/', size='full')

In [4]:
# get only files that have been downloaded

downloaded = d.train.apply(lambda x: os.path.exists(os.path.join(d.path, x.filename[:-4] + '_cropped.mp4')), axis=1)

In [5]:
d.train = d.train[downloaded]

d.train

,filename,url,project_id,num_frames,crowd_score,tier1,micro,nano,stalled
0,100000.mp4,s3://drivendata-competition-clog-loss/train/10...,M,54,0.000000,True,False,False,0
1,100001.mp4,s3://drivendata-competition-clog-loss/train/10...,F,48,0.022769,False,False,False,0
2,100002.mp4,s3://drivendata-competition-clog-loss/train/10...,H,122,0.000000,True,False,False,0
3,100003.mp4,s3://drivendata-competition-clog-loss/train/10...,E,55,0.000000,True,False,False,0
4,100004.mp4,s3://drivendata-competition-clog-loss/train/10...,C,56,0.000000,True,False,False,0
...,...,...,...,...,...,...,...,...,...
570501,684600.mp4,s3://drivendata-competition-clog-loss/train/68...,A,49,1.000000,True,True,True,1
570636,684744.mp4,s3://drivendata-competition-clog-loss/train/68...,G,53,0.000000,True,True,True,0
571469,685597.mp4,s3://drivendata-competition-clog-loss/train/68...,K,48,0.000000,True,True,True,0
571511,685640.mp4,s3://drivendata-competition-clog-loss/train/68...,C,58,0.000000,True,True,False,0


In [6]:
class_weights = len(d.train)/d.train.stalled.value_counts().values/2

In [24]:
def get_n_frames(x):
    import cv2
    
    if os.path.exists(x):
    
        cap = cv2.VideoCapture(x)

        return int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    else:
        return np.nan

In [25]:
n_frames = d.train.apply(lambda x: get_n_frames(os.path.join(d.path, x.filename[:-4] + '_cropped.mp4')), axis=1)

In [7]:
cnn = tf.keras.applications.VGG16()
latent_dim = 1000

In [17]:
# DO SOME QUICK "CROSS VALIDATION" FOR NEURAL NET HYPERPARAMETERS

lstm_units  = [64, 128, 256, 512, 1024]
dense_units = [4, 16, 64, 128, 256, 512, 1024]
repeats = 4

mcc = np.zeros((len(lstm_units), len(dense_units), repeats))
acc = np.zeros((len(lstm_units), len(dense_units), repeats))
history = np.empty((len(lstm_units), len(dense_units), repeats), dtype=object)

for i in range(len(lstm_units)):
    for j in range(len(dense_units)):
        for k in range(repeats):
            model = CNNFeatureLSTM(cnn, cnn_dim=latent_dim, lstm_units=lstm_units[i], dense_units=dense_units[j])

            history[i,j,k] = model.fit(d, epochs=50, callbacks=[EarlyStopping(monitor='val_matthewcorr', 
                                                               patience=5,
                                                               mode='max',
                                                               restore_best_weights=True)], verbose=0)

            best_epoch = np.argmax(history[i,j,k].history['val_matthewcorr'])

            mcc[i,j,k] = history[i,j,k].history['val_matthewcorr'][best_epoch]
            acc[i,j,k] = history[i,j,k].history['val_accuracy'][best_epoch]

LOADING CNN FEATURES...

TRAINING CLASSIFICATION MODEL...
LOADING CNN FEATURES...

TRAINING CLASSIFICATION MODEL...
LOADING CNN FEATURES...

TRAINING CLASSIFICATION MODEL...
LOADING CNN FEATURES...

TRAINING CLASSIFICATION MODEL...
LOADING CNN FEATURES...

TRAINING CLASSIFICATION MODEL...
LOADING CNN FEATURES...

TRAINING CLASSIFICATION MODEL...
LOADING CNN FEATURES...

TRAINING CLASSIFICATION MODEL...
LOADING CNN FEATURES...

TRAINING CLASSIFICATION MODEL...
LOADING CNN FEATURES...

TRAINING CLASSIFICATION MODEL...
LOADING CNN FEATURES...

TRAINING CLASSIFICATION MODEL...
LOADING CNN FEATURES...

TRAINING CLASSIFICATION MODEL...
LOADING CNN FEATURES...

TRAINING CLASSIFICATION MODEL...
LOADING CNN FEATURES...

TRAINING CLASSIFICATION MODEL...
LOADING CNN FEATURES...

TRAINING CLASSIFICATION MODEL...
LOADING CNN FEATURES...

TRAINING CLASSIFICATION MODEL...
LOADING CNN FEATURES...

TRAINING CLASSIFICATION MODEL...
LOADING CNN FEATURES...

TRAINING CLASSIFICATION MODEL...
LOADING CNN FE

InternalError:  [_Derived_]  Failed to call ThenRnnBackward with model config: [rnn_mode, rnn_input_mode, rnn_direction_mode]: 2, 0, 0 , [num_layers, input_size, num_units, dir_count, max_seq_length, batch_size, cell_num_units]: [1, 1000, 512, 1, 100, 32, 512] 
	 [[{{node gradients/CudnnRNN_grad/CudnnRNNBackprop}}]]
	 [[StatefulPartitionedCall]] [Op:__inference_distributed_function_1773773]

Function call stack:
distributed_function -> distributed_function -> distributed_function


In [8]:
model = CNNFeatureLSTM(cnn, cnn_dim=latent_dim, lstm_units=512, dense_units=256)

In [ ]:
history = model.fit(d, epochs=50, callbacks=[EarlyStopping(monitor='val_matthewcorr', 
                                                               patience=8,
                                                               mode='max',
                                                               restore_best_weights=True)])

FINDING CNN FEATURES...
100%|██████████| 322837/322837 [47:05:59<00:00,  1.90it/s]   

TRAINING CLASSIFICATION MODEL...


In [ ]:
model.model.save_weights(os.path.join(d.path, 'vgg16_lstm_512lstm_256dense_val03465.ckpt'))

In [ ]:
predictions = model.predict(d, train=False).flatten()

 39%|███▉      | 5532/14160 [1:01:11<1:43:35,  1.39it/s]

In [23]:
import pandas as pd
df = pd.DataFrame({'filename':d.test.filename.values, 'stalled': (predictions.flatten() > 0.5).astype(int)})

In [24]:
df.to_csv(os.path.join(d.path, 'pred_vgg16_lstm_512lstm_256dense_val03465.csv'), index=False)

In [25]:
print('hi'); x=10

hi


In [26]:
x

10